<!-- TABS -->
# Create Vector Index

In [ ]:
# <testing: MongoDB>
from superduperdb import superduper, Document
from superduperdb.backends.mongodb import Collection


db = superduper('mongomock://temp')

datas = []
for i in range(10):
    data = {
        "my_key": str(i),
    }
    datas.append(data)

db.execute(Collection("documents").insert_many([Document(data) for data in datas]))
select = Collection("documents").find()

In [ ]:
# <testing: >
import numpy as np
from superduperdb.ext.numpy import array
from superduperdb import ObjectModel

model = ObjectModel("test", object=lambda x: np.random.randn(16, ), datatype=array("float64", shape=(16, )))

- `model` is a `_Predictor` which converts input data to `vector`, `array` or `tensor`.
- `select` is a `Select` query telling which data to search over.
- `key` is a `str`, tuple of `str` or `dict` telling the models how to consume the content of `Document` instances.

In [ ]:
# <tab: VectorIndex>
from superduperdb import VectorIndex, Listener

db.add(
    VectorIndex(
        'my-vector-index',
        indexing_listener=Listener(
            key='my_key',      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=model,         # a `_Predictor` how to convert data to embeddings
        )
    )
)

In [ ]:
# <tab: Model.to_vector_index>
db.add(model.to_vector_index(key='my_key', select=select, identifier="my-vector-index"))

In [ ]:
# <testing: >
from superduperdb.backends.mongodb import Collection

collection = Collection('documents')

select = collection.like(
    Document({"my_key": 1}), 
    vector_index="my-vector-index", 
    n=5
).find()
results = db.execute(select)
list(results)